In [0]:
import gradio as gr
def greet(name):
    return "Hello " + name + "!"
def shout(name):
    return name.upper()

In [0]:
#demo = gr.Interface(fn=shout, inputs=gr.Textbox(), outputs=gr.Textbox()) //this works too
demo = gr.Interface(fn=greet, inputs="textbox", outputs="textbox",allow_flagging="never")
demo.launch()

In [0]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import ollama

In [0]:
system_message = "You are a helpful assistant"

In [0]:
def message_llama(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = ollama.chat(
        model='llama3.2',
        messages=messages,
    )
    return completion['message']['content']

In [0]:
message_llama("what is the date today")

In [0]:
def stream_llama(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = ollama.chat(
        model='llama3.2',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk['message']['content'] or ""
        yield result
   

In [0]:
def stream_deepseek(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = ollama.chat(
        model='deepseek-r1',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk['message']['content'] or ""
        yield result

In [0]:
def stream_qwen3(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = ollama.chat(
        model='qwen3',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk['message']['content'] or ""
        yield result

In [0]:
def stream_model(prompt, model):
    if model=="llama3.2":
        result = stream_llama(prompt)
    elif model=="deepseek-r1":
        result = stream_deepseek(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [0]:
gr.Interface(fn=stream_model, inputs=[gr.Textbox(label="Your Message"),gr.Dropdown(["llama3.2", "deepseek-r1"], label="Select model", value="llama3.2")], outputs=[gr.Markdown(label="Response")],flagging_mode="never").launch()

In [0]:
gr.Interface(fn=stream_llama, inputs=[gr.Textbox(label="Your Message")], outputs=[gr.Markdown(label="Response")],flagging_mode="never").launch()

In [0]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [0]:
system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."
# For Fun
tone_description_fun = """
    The tone should be:
    - **Fun and Playful:** Inject humor, use lighthearted language, and maintain an upbeat vibe.
    - **Energetic:** Use active voice, strong verbs, and occasional exclamation points.
    - **Approachable:** Write as if speaking to a friend, using slightly informal language and contractions.
    - **Creative:** Think outside the box for descriptions and calls to action.
    - Avoid sounding childish or overly silly.
"""

# For Aggression
tone_description_aggression = """
    The tone should be:
    - **Bold and Assertive:** Use strong, direct language that conveys confidence and power.
    - **Challenging:** Pose questions that make the reader reconsider their current solutions.
    - **Urgent:** Imply a need for immediate action and emphasize competitive advantages.
    - **Direct and Punchy:** Employ short, impactful sentences and strong calls to action.
    - **Dominant:** Position the company as a leader and a force to be reckoned with.
    - Avoid being rude, offensive, or overly hostile. Focus on competitive intensity.
"""

In [0]:
def stream_brochure(company_name, url, model, tone):
    prompt = f"Please generate a company brochure for {company_name} that embodies the following tone and style guidelines: {tone}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model=="llama":
        result = stream_llama(prompt)
    elif model=="deepseek":
        result = stream_deepseek(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [0]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["llama", "deepseek"], label="Select model"),
        gr.Dropdown(["tone_description_fun", "tone_description_aggression"])],
    outputs=[gr.Markdown(label="Brochure:")],
        
    flagging_mode="never"
)
view.launch()